In [ ]:
"""
이미지 기반 객체 탐지 예제
===================================
이 노트북은 이미지 기반 검색을 사용한 객체 탐지 예제입니다.
각 클래스별로 대표 이미지를 사용하여 유사한 객체를 탐지합니다.

"""

import os
import json
from pathlib import Path

import torch
from torchvision.ops import nms
from transformers import Owlv2Processor, Owlv2ForObjectDetection
from PIL import Image, ImageDraw, ImageFont

# ==================== 디렉토리 설정 ====================
IMAGE_DIR        = Path("encumbrance_labels/encumbrance")           # 원본 이미지 디렉토리
EACH_CLASSES_DIR = Path("encumbrance_labels/each_classes")          # 클래스별 대표 이미지 디렉토리
OUTPUT_DIR       = Path("encumbrance_labels/prediction_with_multiple_images")  # 결과 저장 디렉토리

# ==================== 하이퍼파라미터 ====================
TARGET_SIZE      = 1008   # OWLv2 최적 입력 크기 (14×72 패치)
GUIDED_THRESHOLD = 0.9    # 이미지 기반 매칭 신뢰도 임계값
IOU_THRESHOLD    = 0.1    # NMS IoU 임계값

# ==================== 시스템 설정 ====================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 클래스 이름 자동 감지 (each_classes 디렉토리의 하위 폴더명)
CLASS_NAMES = [p.name for p in EACH_CLASSES_DIR.iterdir() if p.is_dir()]
CLASS_NAMES.sort()
print(f"탐지 대상 클래스: {CLASS_NAMES}")

# ==================== 시각화 색상 설정 ====================
COLOR_MAP = {
    "Building":   (0, 255, 255),  # 청록색 - 건물
    "Container":  (255, 255,   0), # 노란색 - 컨테이너
    "Field":      (255,   0, 255), # 자홍색 - 밭
    "Greenhouse": (255,   0,   0), # 빨간색 - 온실
    "Tomb":       (0,     0, 255),
    "Tree":       (0,   255,   0),
}
# ────────────────────────────────────────────────────

# ==================== 모델 초기화 ====================
# 1) OWLv2 모델 & 프로세서 로드
processor = Owlv2Processor.from_pretrained(
    "google/owlv2-large-patch14-ensemble",
    image_size=TARGET_SIZE  # 1008x1008 입력 크기
)
model = Owlv2ForObjectDetection.from_pretrained(
    "google/owlv2-large-patch14-ensemble"
).to(DEVICE).eval()  # 평가 모드로 설정

# 2) 출력 폴더 만들기
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
font = ImageFont.load_default()  # 텍스트 표시용 폰트

# ==================== 메인 처리 루프 ====================
# 3) 이미지별 처리 - 각 이미지에 대해 이미지 기반 탐지 수행
for img_path in IMAGE_DIR.glob("*"):
    # 지원하는 이미지 형식만 처리
    if img_path.suffix.lower() not in {".jpg", ".jpeg", ".png"}:
        continue

    print(f"\n▶ Processing {img_path.name} …")
    
    # ==================== 이미지 전처리 ====================
    # 원본 이미지 로드 및 RGB 변환
    orig = Image.open(img_path).convert("RGB")
    # OWLv2 모델 입력 크기로 리사이징
    img = orig.resize((TARGET_SIZE, TARGET_SIZE), Image.BILINEAR)
    w, h = img.size  # w == h == 1008

    # ==================== 이미지 기반 탐지 결과 저장용 리스트 ====================
    all_boxes, all_scores, all_labels = [], [], []

    # ==================== 4단계: 클래스별 이미지 기반 탐지 (배치 처리) ====================
    # 각 클래스마다 대표 이미지들을 사용하여 유사한 객체 탐지
    for cls in CLASS_NAMES:
        # 클래스별 대표 이미지 경로들 가져오기
        exemplar_paths = list((EACH_CLASSES_DIR/cls).glob("*"))
        if not exemplar_paths:
            continue

        # ==================== 대표 이미지들 로드 및 전처리 ====================
        # 각 클래스의 대표 이미지들을 1008x1008로 리사이징
        query_images = [
            Image.open(p).convert("RGB").resize((TARGET_SIZE, TARGET_SIZE), Image.BILINEAR)
            for p in exemplar_paths
        ]
        # 텍스트 라벨과 이미지 배치 준비
        texts = [cls] * len(query_images)  # 각 대표 이미지마다 클래스명
        images_batch = [img] * len(query_images)  # 동일한 원본 이미지를 대표 이미지 개수만큼 복제

        print(f"  • Guided detection for class '{cls}' with {len(query_images)} exemplars")

        # ==================== OWLv2 이미지 기반 탐지 입력 준비 ====================
        # 이미지 기반 탐지를 위한 특별한 입력 형식
        inputs = processor(
            images=images_batch,      # 원본 이미지들 (배치)
            text=texts,              # 클래스명 텍스트들
            query_images=query_images,  # 대표 이미지들 (쿼리)
            return_tensors="pt"
        )
        # GPU로 데이터 이동
        inputs = {k: v.to(DEVICE) for k, v in inputs.items() if isinstance(v, torch.Tensor)}

        # ==================== 이미지 기반 탐지 추론 ====================
        # GPU 메모리 사용량 최적화를 위한 no_grad 컨텍스트
        with torch.no_grad():
            # OWLv2의 이미지 기반 탐지 기능 사용
            outputs = model.image_guided_detection(**inputs)
            
        # 이미지 기반 탐지 결과 후처리
        results_batch = processor.post_process_image_guided_detection(
            outputs=outputs,
            target_sizes=torch.tensor([[h, w]] * len(images_batch), device=DEVICE),
            threshold=GUIDED_THRESHOLD  # 이미지 기반 탐지 임계값
        )

        # ==================== 클래스별 탐지 결과 수집 ====================
        # 각 대표 이미지별 탐지 결과를 전체 리스트에 추가
        for res in results_batch:
            for box, score in zip(res["boxes"], res["scores"]):
                all_boxes.append(box)      # 바운딩 박스 좌표
                all_scores.append(score)   # 신뢰도 점수
                all_labels.append(cls)     # 클래스명

    # ==================== 5단계: 텍스트 기반 탐지 폴백 ====================
    # 이미지 기반 탐지에서 아무것도 찾지 못한 경우 텍스트 기반 탐지로 폴백
    if not all_boxes:
        print(f"  [FALLBACK] no guided detections, running text-based zero-shot")
        
        # ==================== 텍스트 기반 탐지 입력 준비 ====================
        txt_inputs = processor(
            text=[CLASS_NAMES],  # 모든 클래스명을 텍스트로
            images=[img],        # 단일 이미지
            return_tensors="pt"
        )
        # GPU로 데이터 이동
        txt_inputs = {k: v.to(DEVICE) for k, v in txt_inputs.items() if isinstance(v, torch.Tensor)}
        
        # ==================== 텍스트 기반 탐지 추론 ====================
        with torch.no_grad():
            txt_out = model(**txt_inputs)  # 일반적인 텍스트 기반 탐지
            
        # 텍스트 기반 탐지 결과 후처리
        txt_res = processor.post_process_object_detection(
            outputs=txt_out,
            target_sizes=torch.tensor([[h, w]], device=DEVICE),
            threshold=0.3  # 텍스트 기반 탐지 임계값
        )[0]
        
        # 텍스트 기반 탐지 결과 처리
        boxes2, scores2, labels2 = txt_res["boxes"], txt_res["scores"], txt_res["labels"]
        # NMS 적용
        keep2 = nms(boxes2, scores2, IOU_THRESHOLD)
        # 폴백 결과를 전체 리스트에 추가
        for idx in keep2:
            all_boxes.append(boxes2[idx])
            all_scores.append(scores2[idx])
            all_labels.append(CLASS_NAMES[int(labels2[idx])])

    # ==================== 6단계: 최종 NMS 및 시각화 ====================
    # 여전히 탐지 결과가 없는 경우 스킵
    if not all_boxes:
        print(f"  [WARN] still no detections, skipping")
        continue

    # 텐서로 변환
    boxes_t  = torch.stack(all_boxes)   # 모든 바운딩 박스를 하나의 텐서로
    scores_t = torch.stack(all_scores)  # 모든 점수를 하나의 텐서로
    
    # 최종 NMS 적용: 겹치는 박스들 중에서 가장 높은 점수의 박스만 유지
    keep     = nms(boxes_t, scores_t, IOU_THRESHOLD)

    # ==================== 시각화 및 JSON 작성 ====================
    draw = ImageDraw.Draw(img)  # 이미지에 그리기 객체 생성
    records = []  # JSON 저장용 결과 리스트

    # NMS로 필터링된 최종 탐지 결과에 대해 시각화
    for i in keep:
        # ==================== 박스 좌표 처리 ====================
        x1, y1, x2, y2 = boxes_t[i].tolist()
        # 이미지 경계를 벗어나는 박스 좌표를 이미지 내부로 제한
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        # 클래스 정보 및 색상 설정
        label = all_labels[i]  # 클래스명
        score = float(scores_t[i])  # 신뢰도 점수
        color = COLOR_MAP.get(label, (255,255,255))  # 클래스별 색상

        # ==================== 바운딩 박스 그리기 ====================
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        
        # ==================== 라벨 텍스트 표시 ====================
        txt = f"{label}: {score:.2f}"  # 클래스명과 신뢰도 점수
        # 텍스트 배경 박스 계산
        tx0, ty0, tx1, ty1 = draw.textbbox((x1, y1), txt, font=font)
        # 텍스트 배경 그리기
        draw.rectangle([tx0, ty0, tx1, ty1], fill=color)
        # 흰색 텍스트 그리기
        draw.text((x1, y1), txt, fill=(255,255,255), font=font)

        # ==================== JSON 기록용 데이터 추가 ====================
        records.append({
            "label": label,
            "score": score,
            "box": [round(x1,2), round(y1,2), round(x2,2), round(y2,2)]
        })

    # ==================== 7단계: 결과 파일 저장 ====================
    out_img  = OUTPUT_DIR / img_path.name  # 시각화된 이미지 저장 경로
    out_json = OUTPUT_DIR / f"{img_path.stem}.json"  # JSON 결과 저장 경로
    
    # 시각화된 이미지 저장
    img.save(out_img)
    # JSON 결과 저장 (UTF-8 인코딩, 한글 지원)
    with open(out_json, "w", encoding="utf-8") as fp:
        json.dump(records, fp, ensure_ascii=False, indent=2)
    print(f"  [OK] Saved → {out_img.name}, {out_json.name}")

/home/dromii4/dromii/owlv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



▶ Processing DJI_0951.JPG …
  • Guided detection for class 'Building' with 5 exemplars
  • Guided detection for class 'Container' with 3 exemplars
  • Guided detection for class 'Field' with 5 exemplars
  • Guided detection for class 'Greenhouse' with 5 exemplars
  • Guided detection for class 'Tomb' with 1 exemplars
  • Guided detection for class 'Tree' with 5 exemplars
  [OK] Saved → DJI_0951.JPG, DJI_0951.json

▶ Processing DJI_0691.JPG …
  • Guided detection for class 'Building' with 5 exemplars
  • Guided detection for class 'Container' with 3 exemplars
  • Guided detection for class 'Field' with 5 exemplars
  • Guided detection for class 'Greenhouse' with 5 exemplars
  • Guided detection for class 'Tomb' with 1 exemplars
  • Guided detection for class 'Tree' with 5 exemplars
  [OK] Saved → DJI_0691.JPG, DJI_0691.json

▶ Processing DJI_0033.JPG …
  • Guided detection for class 'Building' with 5 exemplars
  • Guided detection for class 'Container' with 3 exemplars
  • Guided detec